# 电商常用数据库地址、库名
***  
|Mysql数据库ip|用处|数据库|说明|-u|-p|
|----|----|----|----|----|----|
|192.168.0.227|采集入库待处理|oridata库|基础表|dev|Data227or8Dev715#|
| | |ec_cross_border库|跨境库|
||7个主要平台|oridata_ec_other|国美、苏宁、亚马逊|
|||oridata_jd|京东|
|||oridata_taobao|淘宝|
|||oridata_tmall|天猫|
||生活服务平台|life_service|
|192.168.0.228|数据处理过后存放库|e_commerce|
|120.24.100.34|省厅数据库(数据处理好之后推送到这个库)|elecb_data||root|123456|
|192.168.0.125|项目zjdatasite|zj_data/ec_sql_chart|
|192.168.0.230|天猫注销和地址变更需要|all_pingtai_shopinfo|  

#### 库中表说明:
1. oridata库:company_ad_xxyy 工商地址库,taobao_cat_sales_vxxx最新类目表,company_qichacha_xxyy企查查表,taobao_brandinfo_vxxxx 阿里品牌表,baidu2normal_revise 区县市纠正地址库  
2. ec_cross_border库: alibabagj_xxx　阿里巴巴国际,dhgate_xxx敦煌,ebay_xxx易贝网，lazada_xxx,smt_xxx速卖通
3. hive数据库: jdbc:hive2://192.168.0.81:10000/default，username:hadoop、password:hadoop
4. 项目zjdatasite和192.168.0.125zj_data库: https://192.168.0.125:8443/svn/myrepository/038NriatDataAdmin/trunk/zjdata ,ec_sql_chart表：改hive和228的动态sql

## 一、地址库处理：
***  
1. 月初淘宝、天猫和京东店铺表数据采集完成后，爬虫组（找志才）会采集公司地址信息生成（227）company_ad_20200202(工商地址库,日期随时间变化)和company_qichacha_202001(公司信息库不一定每个月都有),company_ad_diff_xxx(公司地址修正表/公司名称发生变化的表)
    - 运行<kbd><b>工商地址库更新后交省厅.ipynb</b></kbd> ，
    - (即得到工商地址库 company_ad 中新出现的地址，其中为浙江的，且省市区与上月相比发生了变化的,给省厅审查)
    - 得到省厅反馈结果后，将反馈表格放在程序路径里，运行<kbd><b>工商地址更新入库.ipynb</b></kbd> 产生（227）company_ad_st（程序中会自动把上月表格改后缀）
    
    
2. 将227的 company_ad_diff.ipyn 脚本结果传输到192.0.168.228/e_commerce中的 company_ad_diff 表后,运行sql，标准化省市区:
    - （方法一：先在227上运行省市区标准化sql，再跑脚本传数据
    - __update company_ad_diff_202002 t1 left join baidu2normal_revise t2 on t1.province=t2.baidu_province and t1.city= t2.baidu_city and t1.contry = t2.baidu_county set t1.province = t2.normal_province,t1.city=t2.normal_city,t1.contry=t2.normal_county__）
    - 在228上，上个月的company_ad_diff表加上日期后缀改名（无需新建company_ad_diff表）
    - 在227上运行__company_ad_diff.ipynb__（得到发生变动的公司名称中不包含区县名称的给省厅审核，包含区县名称的会被直接导入228相关表）
    - 脚本倒数第一步会自动在228上新建company_ad_diff表，并将数据导入到表中
    - 脚本运行完毕后，得到“新公司地址需审查表”,群里交给婷婷（上个月已经确认过的可以找出来剔除掉）
    - 得到反馈后将结果继续导入228的company_ad_diff中，上月已给省厅确认过的无需再次确认，可做个标记，revise改成1
    - （方法二：数据都导好后，再在228上运行如下省市区标准化sql
    - __update company_ad_diff t1 left join baidu2normal t2 on t1.province=t2.baidu_province and t1.city= t2.baidu_city and t1.contry = t2.baidu_county set t1.province = t2.normal_province,t1.city=t2.normal_city,t1.contry=t2.normal_county__）
    
    company_ad_diff表数据会在第四章的sql处理步骤中用到


3. 将最新的地址库,公司库传输到hadoop,首先上hive修改表名,不然上个月的表会被覆盖掉（-hive的sql必须写库名！-）
    
    （1）修改上月表名为备份表
    - <b>alter table e_commerce.company_ad rename to e_commerce.company_ad_20200305</b>(时间改成当日)
    - __alter table e_commerce.company_qichacha_revise rename to e_commerce.company_qichacha_revise_202002bk__ (本月无需)
    
    （2）导入本月表成为新表
    - __sh sqoopImport.sh "oridata" "company_ad_st" "e_commerce" "company_ad" "192.168.0.227";__
    - __sh sqoopImport.sh "oridata" "company_qichacha_xxxx" "e_commerce" "company_qichacha_revise" "192.168.0.227";__
    （企查查表无更新就不需要导）(本月无需)

## 二、淘宝类目,百度地址库更新:(本月无需)
***
在hive上修改淘宝类目表名并将数据传输到hadoop（无更新则不用处理）
1. __alter table pre_e_commerce.taobao_cat_sale rename to pre_e_commerce.taobao_cat_sale_xxxx__
2. __sh sqoopImport "oridata" "taobao_cat_sale" "pre_e_commerce" "taobao_cat_sale" "192.168.0.227"__
3. __alter table e_commerce.baidu2normal rename to e_commerce.baidu2normal_xxxx__
4. __sh sqoopImport "oridata" "baidu2normal_revise" "e_commerce" "baidu2normal" "192.168.0.227"__

## 三、Hadoop数据处理
***
以后跑之前先检查一下价格异常

-SELECT * FROM tmall_goodsmobile_202003 where sales_month>0 and discount_price like "%9999%" order by discount_price+0 desc

-SELECT * FROM `jd_goodsinfo_202001` where realcomment>0 and price like "%9999%" order by price+0 desc

1. 启动zjdata项目,查看zjdatasite中的Java.Resource.com.nriat.controller.date.__AutoDataConTroller.java__和com.nriat.utils__CmdUtils.java__两个文件中的<kbd>传入</kbd>|<kbd>处理</kbd>|<kbd>输出</kbd>三个模块中的正确性,初始为天猫、淘宝、京东、苏宁、国美五大平台代码未被注释，一号店和亚马逊代码注释掉（亚马逊已不需要）
   - 将要处理的模块代码的注释取消掉，第一步、第二步、第三步全部处理好，点击保存（淘宝企业模块必须有淘宝模块才能跑），运行程序（server工具栏右侧的start the server）
   
2. 程序跑完后，网页登录__http://localhost:8080/zjdatasite/__,账号:1234567890 密码:1111（必须保持登录状态才能进行后续操作），可看到相应表

3. 将227上数据导入hive.pre_e_commerce库,新开一个网页，网页地址处输入:__http://localhost:8080/zjdatasite/AutoData/importData?date=202002__(年月为日期,每个月相应修改),回车执行，执行完毕之后到hive.pre_e_commerce查看,__http://192.168.0.80:8088/cluster__可以查看执行的进度

   （已完成）

4. 上述步骤成功后，处理数据,程序会将hive.pre_e_commerce的数据处理输出至hive.e_commerce,网页地址输入:__localhost:8080/zjdatasite/AutoData/exec?date=202002__,执行完毕之后,在hive.e_commerce查看是否执行成功
    - 数据处理的语句在192.168.0.125数据库中的zj_data库中的ec_sql_chart,如果有需要改变的地方,可以通过修改相应的语句
    
5. 数据输出到228:__localhost:8080/zjdatasite/AutoData/exportData?date=202002__,查看228数据库,数据是否存在（shopinfo表）
   ps: 可以通过对__AutoDataConTroller.java__和__CmdUtils.java__的修改注释,调整数据处理和输入输出的平台数据
    - 京东的数据处理（第4步）经常出现无法进行的error,需要另行处理：
    - 在227oridata 库中,清空jd_goodssales表,右键清空,或者truncate table jd_goodssales
    - 利用xshell导出hive上的 jd_goodssales_xxxx 表至227:__sh sqoopExport.sh "oridata" "jd_goodssales" "pre_e_commerce" "jd_goodssales_202002"  "192.168.0.227"__
    - 把之前第五步失败跑出来的228上的jd_shopinfo_xxxx表删掉，再运行<kbd>京东月数据处理.ipynb</kbd>
    - 结束后将228中生成的京东店铺表导一份到hive，以便下个月计算hiv（今日不做，后续再补）
    - __sh sqoopImport.sh "e_commerce" "jd_shopinfo_202002" "e_commerce" "jd_shopinfo_202002" "192.168.0.228";__

## 四、数据库sql处理
***
__在228的查询中进行，修改月份,以下步骤顺序不要打乱__


1. 淘宝企业添加需要识别的企业店：  
    - ```CREATE table taobao_shopinfo_202002_add_qiye as
    SELECT * from  taobao_shopinfo_202002
    WHERE shop_id in (SELECT shop_id from taobao_shopinfo_supply);```
    - ```Insert into taobao_qiye_shopinfo_202002 SELECT t1.shop_id,t1.seller_id,t1.bc_type,t1.main_sale,t2.company,t1.sales_count,t1.sales_money,t1.address,t1.seller_name,t1.seller_lv,t1.shop_hpl,t1.shop_name,t1.weitao_id,t1.shop_fans_num,t1.open_shop_date,t1.goods_num,t1.new_goods_num,t1.guanzhu_num,t1.shop_money,t1.shop_iocn,t1.gold_shop,t1.shop_type,t1.taobao_shop_type2,t1.taobao_shop_type3,t1.open_year,t1.taobao_tel,t1.company_change,t2.company_address,t2.province,t2.city,t2.county,t2.street,t1.last_sales_money,t1.vix,t2.company_boss,t2.company_tel,t2.shop_url,t1.xinyong,t1.describe_rate,t1.service_rate,t1.logistics_rate
    FROM taobao_shopinfo_202002_add_qiye t1 LEFT JOIN  taobao_shopinfo_supply t2 on t1.shop_Id=t2.shop_id```  


2. 京东主营更新:  
    - ```UPDATE jd_shopinfo_202002 set main_sale_all=cname2,main_sale_top5=cname2,main_sale_count=1 WHERE cname2!='';```


3. 修正企业名称(__company_ad_diff__),运行sql（最好建个索引，速度会快一点）
   改用jupyter程序跑
    - ```update tmall_shopinfo_202002 as t1 , company_ad_diff as t2
set t1.company = t2.companyname,t1.company_address=t2.address,t1.province=t2.province,t1.city=t2.city,t1.county=t2.contry
where t1.company = t2.formername;```
    - ```update jd_shopinfo_202002 as t1 , company_ad_diff as t2
set t1.company = t2.companyname,t1.company_address=t2.address,t1.province=t2.province,t1.city=t2.city,t1.county=t2.contry
where t1.company = t2.formername;```
    - ```update taobao_qiye_shopinfo_202002 as t1 , company_ad_diff as t2
set t1.company = t2.companyname,t1.company_address=t2.address,t1.province=t2.province,t1.city=t2.city,t1.county=t2.contry
where t1.company = t2.formername;```
    - ```update suning_shopinfo_202002 as t1 , company_ad_diff as t2
set t1.company = t2.companyname,t1.company_address=t2.address,t1.province=t2.province,t1.city=t2.city,t1.county=t2.contry
where t1.company = t2.formername;```
    - ```update gome_shopinfo_202002 as t1 , company_ad_diff as t2
set t1.company = t2.companyname,t1.company_address=t2.address,t1.province=t2.province,t1.city=t2.city,t1.county=t2.contry
where t1.company = t2.formername;```


4. 修正企业地址,将__correct_address__的新增数据插入到__company_ad_revise__表中,修正各平台企业地址运行sql
    - ```INSERT into company_ad_revise(company_name,company_ad,province,city,county,street)
    select company as company_name,correct_address as company_ad,IFNULL(correct_province,'浙江省') as province,correct_city as city,correct_county  as county,'' as street
    from correct_address where `status`='1' and company not in (SELECT company_name from company_ad_revise)```
    - ```update tmall_shopinfo_202002 as t1,company_ad_revise as t2 set t1.company_address=t2.company_ad,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county where t1.company=t2.company_name;```
    - ```update taobao_qiye_shopinfo_202002 as t1,company_ad_revise as t2 set t1.company_address=t2.company_ad,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county where t1.company=t2.company_name;```
    - ```update jd_shopinfo_202002 as t1,company_ad_revise as t2 set t1.company_address=t2.company_ad,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county where t1.company=t2.company_name;```
    - ```update suning_shopinfo_202002 as t1,company_ad_revise as t2 set t1.company_address=t2.company_ad,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county where t1.company=t2.company_name;```
    - ```update gome_shopinfo_202002 as t1,company_ad_revise as t2 set t1.company_address=t2.company_ad,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county where t1.company=t2.company_name;```


5. 第二次修正企业名称(__correct_company__),运行sql  
    - ```update tmall_shopinfo_202002 as t1,correct_company as t2 set t1.company = t2.company,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county,t1.company_address=t2.company_address where t1.company=t2.former_company;```
    - ```update taobao_qiye_shopinfo_202002 as t1,correct_company as t2 set t1.company = t2.company,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county,t1.company_address=t2.company_address where t1.company=t2.former_company;```
    - ```update jd_shopinfo_202002 as t1,correct_company as t2 set t1.company = t2.company,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county,t1.company_address=t2.company_address where t1.company=t2.former_company;```
    - ```update gome_shopinfo_202002 as t1,correct_company as t2 set t1.company = t2.company,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county,t1.company_address=t2.company_address where t1.company=t2.former_company;```
    - ```update suning_shopinfo_202002 as t1,correct_company as t2 set t1.company = t2.company,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county,t1.company_address=t2.company_address where t1.company=t2.former_company;```


6. 修正天猫、淘宝企业店铺的企业及地址（ __correct_shop_address_tmall__）,运行sql:
    - ```update tmall_shopinfo_202002 as t1,correct_shop_address_tmall as t2 set t1.company = t2.correct_company,t1.province=t2.correct_province,t1.city=t2.correct_city,t1.county=t2.correct_county,t1.street=t2.correct_street,t1.company_address=t2.correct_company_address where t1.shop_id=t2.shop_id and t2.pingtai='tmall';```
    - ```update taobao_qiye_shopinfo_202002 as t1,correct_shop_address_tmall as t2 set t1.company = t2.correct_company,t1.province=t2.correct_province,t1.city=t2.correct_city,t1.county=t2.correct_county,t1.street=t2.correct_street,t1.company_address=t2.correct_company_address where t1.shop_id=t2.shop_id and t2.pingtai='taobao_qiye';```


7. 修正京东的企业及地址(__correct_shop_address_jd__),运行sql:
    - ```update jd_shopinfo_202002 as t1,correct_shop_address_jd as t2 set t1.company = t2.correct_company,t1.province=t2.correct_province,t1.city=t2.correct_city,t1.county=t2.correct_county,t1.street=t2.correct_street,t1.company_address=t2.correct_company_address where t1.shop_id=t2.shop_id;```


8. 修正苏宁的信息(__company_ad_formatted_address__),运行sql:
    - ```update suning_shopinfo_202002 as t1,company_ad_formatted_address as t2 set t1.company_address=t2.formatted_address,t1.province=t2.province,t1.city=t2.city,t1.county=t2.county
where t1.company=t2.company and (t1.company_address='' or t1.company_address is null);```


9. 运行程序:<kbd>228天猫-主营细分.exe</kbd>（几小时）

## 数据查看
1. 去除浙江省的店铺中数据明显异常的店铺,去除的店铺会保存在__tmall_shopinfo_drop__,__taobao_shopinfo_drop__,__taobao_qiye_shopinfo_drop__,__jd_shopinfo_drop__表（228）中:
    - 运行<kbd>月数据查错.ipynb</kbd>,第一步"浙江省数据查错"得到各平台浙江省销售额较高,商品单价最大的一百间店铺的excel表格,在表格中有商铺链接(此处会需要修改淘宝的url),大概了解商铺中商品的成交价格与表格中显示的单价差异过大,则删除店铺(在表中excel,sign=1),保存。继续运行第二步"数据库纠错"，会将店铺表中的数据插入到drop表。
2. 运行统计程序流程,得到结果（叫志成统计）
3. 结果出来之后，运行<kbd>区县结果对比.ipynb</kbd>,看到本月和上月的浙江省各区县销售额、活跃店铺数据对比,异常区县查找原因

## 推送省厅
1. 将227,oridata_1688中本月的店铺数据（a1688_shopinfp_202002）传输到228,e_commerce
2. 运行一下四个sql语句,生成_cp结尾的数据表


- （1）淘宝企业
    
    - ```create table taobao_qiye_shopinfo_202002_cp as 
    select shop_id,seller_id,seller_name,concat('shop',shop_id,'.taobao.com') as shop_url,shop_name,
    open_shop_date,address,goods_num,new_goods_num,main_sale,sales_money,sales_count,company,company_address,province,city,county,'202002' as collect_month,
    company_boss,company_tel,xinyong,'' as farm_flag
    ,shop_hpl, describe_rate, service_rate, logistics_rate
    from taobao_qiye_shopinfo_202002;```

    - ```ALTER TABLE taobao_qiye_shopinfo_202002_cp MODIFY COLUMN farm_flag VARCHAR(10)  NOT NULL DEFAULT ''  COMMENT '农产品标志';```
    - ```update taobao_qiye_shopinfo_202002_cp t set t.farm_flag = '是' where t.main_sale = '食品/保健' and 
    (locate('茶',shop_name)>0
    or locate('蜂',t.shop_name)>0
    or locate('农',t.shop_name)>0
    or locate('花',t.shop_name)>0
    or locate('畜',t.shop_name)>0
    or locate('食品',t.shop_name)>0
    or locate('特产',t.shop_name)>0
    or locate('火腿',t.shop_name)>0
    or locate('农产品',t.shop_name)>0
    or locate('果',t.shop_name)>0
    or locate('水产',t.shop_name)>0 );```

    - ```ALTER TABLE taobao_qiye_shopinfo_202002_cp ENGINE=MyISAM;``` 
    
    
- （2）天猫
    
    - ```create  table tmall_shopinfo_202002_cp
    select shop_id,seller_id,seller_name,shop_url,shop_name,address,
    open_shop_date,shop_phone,goods_num,new_goods_num,main_sale,sales_money,sales_count,company,company_address,province,city,county,'202002' as collect_month,
    company_boss,company_tel,xinyong,'' as farm_flag
    ,shop_hpl, describe_rate, service_rate, logistics_rate,
    main_sale_count,main_sale_all,main_sale_top5
    from tmall_shopinfo_202002;```
    
    - ```ALTER TABLE tmall_shopinfo_202002_cp MODIFY COLUMN farm_flag VARCHAR(10)  NOT NULL DEFAULT ''  COMMENT '农产品标志';```
    
    - ```update tmall_shopinfo_202002_cp t set t.farm_flag = '是' where t.`main_sale` = '食品/保健' and 
    (locate('茶',t.`shop_name`)>0
    or locate('蜂',t.`shop_name`)>0
    or locate('农',t.`shop_name`)>0
    or locate('花',t.`shop_name`)>0
    or locate('畜',t.`shop_name`)>0
    or locate('食品',t.`shop_name`)>0
    or locate('特产',t.`shop_name`)>0
    or locate('火腿',t.`shop_name`)>0
    or locate('农产品',t.`shop_name`)>0
    or locate('果',t.`shop_name`)>0
    or locate('水产',t.`shop_name`)>0 );```
    
    - ```ALTER TABLE tmall_shopinfo_202002_cp ENGINE=MyISAM;```
    
    
- （3）京东
    
    - ```create  table jd_shopinfo_202002_cp
    select shop_id,shop_name,sales_money,main_sale,open_shop_date,venderid,'' as tel,company,company_address,province,city,county,data_month as collect_month,
    company_boss,company_tel,'' as xinyong,'' as farm_flag
    ,'' as shop_hpl, pingjia_num as describe_rate, fuwu_num as service_rate, wuliu_num as logistics_rate,shop_url,main_sale_count,main_sale_all,main_sale_top5
    from jd_shopinfo_202002;```

    - ```ALTER TABLE jd_shopinfo_202002_cp MODIFY COLUMN farm_flag VARCHAR(10)  NOT NULL DEFAULT ''  COMMENT '农产品标志';```

    - ```update jd_shopinfo_202002_cp  t set t.farm_flag = '是' where t.`main_sale` = '食品饮料' and 
    (locate('茶',t.`shop_name`)>0
    or locate('蜂',t.`shop_name`)>0
    or locate('农',t.`shop_name`)>0
    or locate('花',t.`shop_name`)>0
    or locate('畜',t.`shop_name`)>0
    or locate('食品',t.`shop_name`)>0
    or locate('特产',t.`shop_name`)>0
    or locate('火腿',t.`shop_name`)>0
    or locate('农产品',t.`shop_name`)>0
    or locate('果',t.`shop_name`)>0
    or locate('水产',t.`shop_name`)>0 );```

    - ```ALTER TABLE jd_shopinfo_202002_cp ENGINE=MyISAM;```


- （4）淘宝
    
    - ```create table taobao_shopinfo_202002_cp as 
    select shop_id,seller_id,seller_name,concat('shop',shop_id,'.taobao.com') as shop_url,shop_name,
    open_shop_date,address,goods_num,new_goods_num,main_sale,sales_money,sales_count,company,'202002' as collect_month
    ,shop_hpl, describe_rate, service_rate, logistics_rate
    from taobao_shopinfo_202002;```

    - ```ALTER TABLE taobao_shopinfo_202002_cp ENGINE=MyISAM;```


3. 得到__tmall_address_change_shopinfo_xxxx__和__tmall_logout_shop_xxxx__:（需要用到230数据库）

  将228的__tmall_shopinfo_xxxx_cp__,__jdshopinfo_xxxx_cp__,__taobao_qiye_shopinfo_xxxx_cp__传输到192.168.0.230.all_pingtai_shopinfo中,运行sql:
    - __insert into tmall_shopinfo select * from tmall_shopinfo_202002_cp;__
    - __insert into taobao_qiye_shopinfo select * from taobao_qiye_shopinfo_202002_cp;__
    - __insert into jd_shopinfo select * from jd_shopinfo_202002_cp;__
    - 通知相关人员(目前是志成得到address_change和logout表)
    - 1月数据省市区有进行修正，未更新到230
    
    
4. 推送到省厅库（结果表确认过之后再推送）
    - 打开mstsc远程192.168.0.232:9512(账户:administrator,密码:`nriatp1uimFsQ`（这是123的1），前后有“`”,键盘左上角esc下面的那个),打开桌面软件:__数据库推送__
        - 在方框中输入:__server=192.168.0.228;port=3306;user id=dev;password=Data227or8Dev715#;database=e_commerce;__,下方方框中227改成228，连接数据库
        - 在侧边栏中勾选所需的7张，点右下角推送表:tmall_shopinfo_xxx_cp,taobao_shopinfo_xxxx_cp,taobao_qiye_shopinfo_xxxx_cp,jd_shopinfo_xxxx_cp,suning_shopinfo_xxxx,gome_shopinfo_xxxx,tmall_address_change_shopinfo_xxxx,tmall_logout_shop_xxxx
    - 点击推送会将数据先放到[E:\数据推送\FTP],注意传输类型为MyISAM,再传输ftp传输到__120.24.100.34__的[D:\FileZilla_Sever\TEMP],传输完成后E盘里的项目会消失。
    - 这里偶尔会有推送失败的情况,可能是因为两个路径的登录凭据没有保存：
    - 1.在文件夹路径中输入:__\\192.168.0.228\MysqlDB__,用户名:Administrator,密码:nriatNX3Wdzkm。
    - 2.在文件夹输入：ftp://112.16.5.62:8999/，用户名：data_to_data，密码：data_to_data
    
    - 打开mstsc远程120.24.100.34(账户:administrator,密码:zJsw2018),打开数据库推送程序，选择接收。等待文件传输完毕,打开数据库,在locolhost中的elecb_data,进行数据插入sql:京东,天猫,淘宝企业存入总表,淘宝因为数据量大,按照半年来存储,上半年存入后缀yyyy_01_06,下半年存入后缀yyyy_07_12,1688数据存入按照年份,后缀yyyy_01_12
        - 京东__insert into jd_shopinfo Select * from  jd_shopinfo_202002_cp;__
        - 天猫__insert into tmall_shopinfo Select * from  tmall_shopinfo_202002_cp;__
        - 天猫注销__insert into tmall_logout_shop SELECT * from tmall_logout_shop_202002;__
        - 天猫地址变更__insert into tmall_address_change_shopinfo SELECT * from tmall_address_change_shopinfo_202002;__
        - 淘宝企业__insert into taobao_qiye_shopinfo  Select * from  taobao_qiye_shopinfo_202002_cp;__
        - 淘宝上半年__insert into taobao_shopinfo_2020_01_06  Select * from  taobao_shopinfo_202002_cp;__
        - 淘宝下半年__insert into taobao_shopinfo_2020_07_12  Select * from  taobao_shopinfo_2020xx_cp;__
        - 1688__insert into a1688_shopinfo_2020_01_12(seller_id_en,company,cxt_year,company_id,fans_num,month_sales_count,month_sales_num,month_sales_month,businsee_model,company_area,main_product,goods_num,satisfaction,contacts,contacts_ad,reg_number,legal_person,company_type,lat,lng,phone1,phone2,collect_monthfrom) select seller_id_en,company,cxt_year,company_id,fans_num,month_sales_count,month_sales_num,month_sales_month,businsee_model,company_area,main_product,goods_num,satisfaction,contacts,contacts_ad,reg_number,legal_person,company_type,lat,lng,phone1,phone2,'202002' as collect_month from a1688_shopinfo_202002;__
        
        
 程序都结束之后，省厅那边也没啥问题，然后运行平均成交单价和评分终极版，改日期